## Compressed model performance benchmarking with GuideLLM

This notebook evaluates the system-level performance of the **compressed model**. It uses GuideLLM, an open-source benchmarking tool designed to measure the performance of large language models deployed with **vLLM**. You can use the results of this notebook to understand how compression affects latency, throughput, and scalability compared to the uncompressed baseline. 

**Goal**

Assess the performance efficiency of the compressed model and quantify the gains or trade-offs introduced by compression under realistic serving conditions.

**Key actions**:

- Start a vLLM server to host the compressed model.
  
- Check GPU vRAM.

- Collect metrics such as token throughput, Time to First Token (TTFT), Inter-Token Latency (ITL), and End-to-end request latency.

- Save results in JSON format.

**Outcome**:

- Performance metrics for the compressed model.

- Confidence that the model is good enough in terms of performance.
---

For details on system level performance benchmarking and GuideLLM, see [Performance Benchmarking with GuideLLM](../docs/System_Level_Performance_Benchmarking.md).

### Install dependencies

In [ ]:
# if you did not install dependencies in 02_Base_Performance_Benchmarking/Base.ipynb, uncomment the following line to install dependencies
# !pip install -qqU .

In [ ]:
import os

from guidellm.benchmark import GenerativeBenchmarksReport
from utils import generate, stream

### Check GPU memory

To make sure that you have enough GPU memory to run this notebook:

1. In a terminal window, run the `nvidia-smi` command.

2. If there are processes that are using GPU memory that this notebook requires, run the `kill -9 <pid>` command for each process to stop it.

### Launch an inference server (vLLM) for the compressed model

Set up a vLLM inference server to host the compressed model and to expose an OpenAI-compatible API endpoint. This server is required so that GuideLLM can benchmark system-level performance such as throughput, latency, and time-to-first-token. In Module 6, you can compare the performance benchmarks of the base and compressed models.

**Resources used** : 46GB L40S GPU x 1

For more information about using vLLM, see [Launch Inference Servers for the Base and Compressed Models using vLLM](../docs/Model_Serving_vLLM.md).

####  Set up environment variables

In [ ]:
# set the logging level for vLLM inference
os.environ["VLLM_LOGGING_LEVEL"] = "DEBUG"

#### Run the vLLM server

This example uses the configuration for a single-node, single-GPU set up to launch a vLLM server for the compressed model. 

**NOTE:** The configuration for serving the compressed model and the base model (in the [Base.ipynb](../02_Base_Performance_Benchmarking/Base.ipynb) notebook) is similar, differing by the model name and port.

1. Open a terminal window.
   
2. Make sure that the working directory is the `05_Compressed_Performance_Benchmarking` directory.
   
   `pwd`
   
3. Run the following command to serve the base model by using vLLM:

```bash
vllm serve \
  "../compressed_model/RedHatAI-Llama-3.1-8B-Instruct-int8-dynamic" \
  --host 127.0.0.1 \
  --port 8001 \
  --gpu-memory-utilization 0.6 \
  --tensor-parallel-size 1 \
  --pipeline-parallel-size 1 \
  --max-model-len 2048
```

4. Wait for the server to start, as indicated by the following status updates:

    ```INFO:     Started server process [166518]```\
    ```INFO:     Waiting for application startup.```\
    ```INFO:     Application startup complete.```

**NOTE:** You can safely ignore the following warning message:

`The tokenizer you are loading from '../base_model/RedHatAI-Llama-3.1-8B-Instruct' with an incorrect regex pattern... This will lead to incorrect tokenization.`

#### Verify that the vLLM server is accessible

Use the  **generate** helper function (defined in [utils.py](./utils.py)) to simplify sending requests to the locally-served vLLM model.

This function wraps the OpenAI-compatible Chat Completions API exposed by vLLM.

In [ ]:
compressed_model_path = (
    "../compressed_model/RedHatAI-Llama-3.1-8B-Instruct-int8-dynamic"
)

In [ ]:
# For non streaming results
response = generate(
    model=compressed_model_path,
    prompt="What is photosynthesis?",
    host="127.0.0.1",
    port=8001,
    api_key="empty",
    max_tokens=512,
)
print(response)

In [ ]:
# For streaming results
res = ""
for chunk in stream(
    model=compressed_model_path,
    prompt="What is photosynthesis?",
    host="127.0.0.1",
    port=8001,
    api_key="empty",
    max_tokens=512,
):
    res += chunk
    print(chunk, end="", flush=True)

### Check GPU vRAM

Loading the compressed model with the configuration defined in the previous command uses approximately 28GB. It might seem surprising that a compressed 8.5 GB model consumes approximately 28GB of GPU memory. This consumption is expected for vLLM because of how memory is allocated during inference due to the following contributors:

* **Model Weights (~8.5 GB)**

    The size of your compressed model stored on disk (INT8, FP16, etc.) which is loaded once into GPU memory.
   
* **Runtime GPU Memory (~6 GB)**

  vLLM reserves extra memory for:

   - Parameter sharding.

   - CUDA kernels.

   - Attention buffers and temporary tensors.

   - Weight adapters and padded tensors which adds ~4–8 GB, depending on the model.

* **KV Cache (~14 GB)**

   - Stores key/value tensors for each generated token to avoid recomputation.

   - Memory grows with sequence length, model hidden size, and concurrency.

   - vLLM presets a large KV cache to support batching efficiently.


* **GPU Memory Utilization Flag (--gpu-memory-utilization)**

``--gpu-memory-utilization`` is set to 0.6, which means that vLLM can utilize 60% of the total GPU memory. This example uses one 46GB LS40 GPU. 60% of 46 is approximately 28.

### Run performance benchmarking

After you verify that the vLLM server for the compressed model is running, you can proceed with benchmarking the compressed model's performance by using **GuideLLM**.

1. Identify values for the following parameters:

   - **target**: The URL of the vLLM inference server started in the previous step, for example: `http://127.0.0.1:8000` If the vLLM server is running on a different port, update the `target` accordingly.

   - **output-path**: The path to save the benchmarking results.

2. Open a terminal window.
   
4. Make sure that the working directory is the `02_Base_Performance_Benchmarking` directory.

   `pwd`
   
6. Check the following command and, if needed, edit the **target** and **output-path** parameters. Then run the command in the terminal.

```bash
guidellm benchmark \
  --target "http://127.0.0.1:8001" \
  --profile sweep \
  --max-seconds 120 \
  --data "prompt_tokens=1024,output_tokens=512" \
  --output-path "../results/compressed_performance_benchmarks.json"
```

#### View the results

The result of the `guidellm benchmark` command is formatted in multiple tables. The results display in the terminal and are saved to the directory defined by `output-path`.

1. **Request Latency Statistics (Completed Requests)**

This table focuses on how **long** requests take and the latency characteristics of the server.

```text
ℹ Request Latency Statistics (Completed Requests)
|=============|=========|========|=========|=========|======|======|=======|=======|
| Benchmark   | Request Latency || TTFT             || ITL        || TPOT         ||
| Strategy    | Sec             || ms               || ms         || ms           ||
|             | Mdn     | p95    | Mdn     | p95     | Mdn  | p95  | Mdn   | p95   |
|-------------|---------|--------|---------|---------|------|------|-------|-------|
| synchronous | 7.6     | 7.9    | 87.9    | 445.7   | 14.7 | 14.7 | 14.8  | 15.5  |
| throughput  | 70.4    | 74.8   | 36149.3 | 40360.5 | 63.9 | 99.4 | 137.4 | 146.1 |
| constant    | 8.3     | 8.3    | 99.4    | 108.0   | 16.1 | 16.1 | 16.2  | 16.3  |
| constant    | 8.9     | 8.9    | 99.1    | 107.2   | 17.2 | 17.3 | 17.4  | 17.4  |
| constant    | 9.7     | 9.8    | 104.4   | 113.0   | 18.8 | 18.9 | 19.0  | 19.1  |
| constant    | 10.5    | 10.6   | 104.9   | 114.6   | 20.4 | 20.5 | 20.6  | 20.6  |
| constant    | 11.7    | 11.8   | 106.9   | 118.1   | 22.7 | 22.8 | 22.8  | 23.0  |
| constant    | 12.7    | 12.8   | 108.3   | 119.3   | 24.7 | 24.8 | 24.9  | 24.9  |
| constant    | 16.0    | 18.5   | 121.6   | 959.9   | 31.1 | 34.7 | 31.3  | 36.1  |
| constant    | 17.8    | 18.1   | 119.7   | 136.0   | 34.5 | 35.2 | 34.7  | 35.4  |
|=============|=========|========|=========|=========|======|======|=======|=======|

```

2.  **Server Throughput Statistics**

This table focuses on how many requests a server can handle per second. Throughput can be thought of as the **rate** (or time required) of processing. 
```text
Server Throughput Statistics
|=============|=====|======|=======|=======|========|========|=======|========|=======|========|
| Benchmark   | Requests                |||| Input Tokens   || Output Tokens || Total Tokens  ||
| Strategy    | Per Sec   || Concurrency  || Per Sec        || Per Sec       || Per Sec       ||
|             | Mdn | Mean | Mdn   | Mean  | Mdn    | Mean   | Mdn   | Mean   | Mdn   | Mean   |
|-------------|-----|------|-------|-------|--------|--------|-------|--------|-------|--------|
| synchronous | 0.1 | 0.1  | 1.0   | 1.0   | 139.6  | 148.9  | 68.2  | 67.6   | 68.2  | 207.3  |
| throughput  | 0.6 | 2.6  | 194.0 | 152.8 | 123.1  | 4262.7 | 966.7 | 1369.8 | 971.8 | 4200.5 |
| constant    | 0.4 | 0.4  | 4.0   | 3.3   | 456.2  | 465.9  | 217.6 | 209.7  | 217.8 | 643.0  |
| constant    | 0.7 | 0.7  | 6.0   | 6.1   | 779.7  | 789.8  | 326.7 | 353.9  | 327.1 | 1085.1 |
| constant    | 1.0 | 1.0  | 10.0  | 9.3   | 1103.9 | 1113.8 | 422.1 | 495.3  | 422.3 | 1518.7 |
| constant    | 1.3 | 1.2  | 14.0  | 12.8  | 1426.6 | 1437.8 | 498.8 | 634.7  | 499.5 | 1946.3 |
| constant    | 1.7 | 1.5  | 19.0  | 17.3  | 1753.6 | 1761.9 | 629.9 | 770.0  | 630.6 | 2361.2 |
| constant    | 2.0 | 1.7  | 25.0  | 22.0  | 2078.6 | 2085.8 | 746.2 | 901.6  | 747.0 | 2764.8 |
| constant    | 2.3 | 2.0  | 36.0  | 32.4  | 2401.2 | 2674.7 | 783.9 | 1110.9 | 786.0 | 3406.4 |
| constant    | 2.5 | 2.2  | 44.0  | 37.5  | 2733.0 | 2733.5 | 829.7 | 1123.5 | 831.5 | 3445.2 |
|=============|=====|======|=======|=======|========|========|=======|========|=======|========|


```
#### Compressed Model Performance Summary
1. Max concurrency under load: 44.0 (Concurrency Mdn)
2. Max output tokens per second under load: 829.7 (Output tokens per sec Mdn)
3. Request latency under load: 17.8 (Request Latency in secs Mdn)
4. Time to first token under load: 119.7 (TTFT ms Mdn)
5. Inter token latency under load: 34.5 (ITL ms Mdn)


#### SLO Analysis

Assume the Service Level Objective (SLO) is:

    TTFT ≤ 200 milliseconds for 95% of requests (p95) with optimal concurrency

At the highest tested concurrency of **44 requests**, the compressed model achieves a **p95 TTFT of 136.0 ms**, which comfortably satisfies the SLO.

This configuration meets the TTFT SLO of 200 ms for 95% of requests. Increasing concurrency beyond this point may push p95 TTFT above the SLO threshold and should be evaluated carefully in production scenarios.

For a workload of **1024 input tokens and 512 output tokens**, the system can sustain approximately **37–44 concurrent requests** while remaining within the TTFT ≤ 200 ms SLO. Reducing input and output token lengths (e.g., 512/256) allows the system to support more concurrent requests while maintaining compliance with the SLO.

#### Comparison with Base Model Performance

| Metric | Base Model | Compressed Model |
|------|-----------|------------------|
| p95 TTFT (ms) | 162.4 ms | **136.0 ms** |
| Max concurrency under SLO | 34 requests | **44 requests** |
| SLO satisfied | Yes | Yes |


In [ ]:
# Run this cell after the benchmarking process in the terminal completes
report = GenerativeBenchmarksReport.load_file(
    path="../results/compressed_performance_benchmarks.json",
)
compressed_benchmarks = report.benchmarks

In [ ]:
compressed_benchmarks[0]